In [1]:
import pandas as pd
import cvxpy as cp
import numpy as np
from numpy import random

# File path will be different on another device and must be replaced with the correct file path.
students = pd.read_csv("C:/Users/Morgan Eads/Documents/ISE 3230/Sample Data (for scheduling) - Students.csv")
print(students.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Student ID       60 non-null     int64 
 1   First Name       60 non-null     object
 2   Last Name        60 non-null     object
 3   Gender           60 non-null     object
 4   Age              60 non-null     int64 
 5   Times Available  60 non-null     object
dtypes: int64(2), object(4)
memory usage: 2.9+ KB
None


In [2]:
random.seed(3230)
num_teachers = 14 # Say we have 14 available teachers in total.
ids = list(range(0, num_teachers))
num_elem = 0
grade = []
for i in range(num_teachers):
    g = random.choice(["elem", "ms"])
    if (num_elem < 5): # Say we have 5 of those teachers are elementary teachers only.
        grade.append(g)
        if (g == "elem"):
            num_elem = num_elem + 1
    else:
        grade.append("ms")     
certificate = random.choice([0,1], size=num_teachers)
teachers = pd.DataFrame({'Teacher ID': ids, 'Grade': grade, 'Certificate': certificate})
display(teachers)

,Teacher ID,Grade,Certificate
0,0,elem,1
1,1,elem,0
2,2,ms,1
3,3,ms,0
4,4,elem,1
5,5,elem,0
6,6,elem,0
7,7,ms,1
8,8,ms,0
9,9,ms,0


In [3]:
students['Age'].value_counts()

Age
9     13
12    12
11    11
13     9
8      8
10     7
Name: count, dtype: int64

In [4]:
# Reference: https://www.geeksforgeeks.org/create-a-new-column-in-pandas-dataframe-based-on-the-existing-columns/
def grade(age):
    if age > 11:
        return "elem"
    else:
        return "ms"

students['Grade'] = students['Age'].map(grade)
display(students.head())

,Student ID,First Name,Last Name,Gender,Age,Times Available,Grade
0,0,Ximeno,Colombo,M,9,"[1, 2, 3, 4]",ms
1,1,Dorothea,Clayton,F,9,"[1, 3, 4]",ms
2,2,Beibhinn,Nye,F,13,"[1, 2, 3]",elem
3,3,Oskar,Ciobanu,M,12,"[1, 2, 3, 4]",elem
4,4,Matilda,Solomon,F,11,"[2, 3]",ms


In [5]:
firstHour = []
secondHour = []
thirdHour = []
fourthHour = []

for i in students['Times Available']:
    x = []
    for j in i:
        if j.isdigit(): # Reference: https://www.w3schools.com/python/ref_string_isdigit.asp
            x.append(int(j))
    if 1 in x:
        firstHour.append(1)
    else:
        firstHour.append(0)
    if 2 in x:
        secondHour.append(1)
    else:
        secondHour.append(0)
    if 3 in x:
        thirdHour.append(1)
    else:
        thirdHour.append(0)
    if 4 in x:
        fourthHour.append(1)
    else:
        fourthHour.append(0)

In [6]:
# Reference: https://www.geeksforgeeks.org/add-multiple-columns-to-dataframe-in-pandas/
students['First Hour'] = firstHour
students['Second Hour'] = secondHour
students['Third Hour'] = thirdHour
students['Fourth Hour'] = fourthHour

In [7]:
display(students.head())

,Student ID,First Name,Last Name,Gender,Age,Times Available,Grade,First Hour,Second Hour,Third Hour,Fourth Hour
0,0,Ximeno,Colombo,M,9,"[1, 2, 3, 4]",ms,1,1,1,1
1,1,Dorothea,Clayton,F,9,"[1, 3, 4]",ms,1,0,1,1
2,2,Beibhinn,Nye,F,13,"[1, 2, 3]",elem,1,1,1,0
3,3,Oskar,Ciobanu,M,12,"[1, 2, 3, 4]",elem,1,1,1,1
4,4,Matilda,Solomon,F,11,"[2, 3]",ms,0,1,1,0


In [8]:
random.seed(3230)
trouble = random.choice([1, 2, 3, 4, 5, 6, 7, 8, 9, 10], size=len(students)) # Reference: https://www.w3schools.com/python/numpy/numpy_random.asp
students['Trouble'] = trouble # Reference: https://www.geeksforgeeks.org/adding-new-column-to-existing-dataframe-in-pandas/

teachers['Relief'] = random.choice([5, 6, 7, 8, 9, 10], size=len(teachers))

In [9]:
display(students.head())

,Student ID,First Name,Last Name,Gender,Age,Times Available,Grade,First Hour,Second Hour,Third Hour,Fourth Hour,Trouble
0,0,Ximeno,Colombo,M,9,"[1, 2, 3, 4]",ms,1,1,1,1,10
1,1,Dorothea,Clayton,F,9,"[1, 3, 4]",ms,1,0,1,1,10
2,2,Beibhinn,Nye,F,13,"[1, 2, 3]",elem,1,1,1,0,7
3,3,Oskar,Ciobanu,M,12,"[1, 2, 3, 4]",elem,1,1,1,1,6
4,4,Matilda,Solomon,F,11,"[2, 3]",ms,0,1,1,0,7


In [10]:
display(teachers.head())

,Teacher ID,Grade,Certificate,Relief
0,0,elem,1,6
1,1,elem,0,10
2,2,ms,1,5
3,3,ms,0,6
4,4,elem,1,10


In [11]:
# Reference: https://www.geeksforgeeks.org/split-pandas-dataframe-by-column-value/
elementary = students[students['Grade'] == "elem"]
middle = students[students['Grade'] == "ms"]
# Reference: https://www.geeksforgeeks.org/python-delete-rows-columns-from-dataframe-using-pandas-drop/
elementary = elementary.drop(['Gender', 'Age', 'Times Available', 'Grade'], axis = 1)
middle = middle.drop(['Gender', 'Age', 'Times Available', 'Grade'], axis = 1)

In [12]:
display(elementary.head())

,Student ID,First Name,Last Name,First Hour,Second Hour,Third Hour,Fourth Hour,Trouble
2,2,Beibhinn,Nye,1,1,1,0,7
3,3,Oskar,Ciobanu,1,1,1,1,6
6,6,Liupold,Gunnarsson,1,1,0,1,9
7,7,Venancio,Vanev,1,1,1,0,8
10,10,Jayadeva,Abbas,1,1,1,1,5


In [13]:
display(middle.head())

,Student ID,First Name,Last Name,First Hour,Second Hour,Third Hour,Fourth Hour,Trouble
0,0,Ximeno,Colombo,1,1,1,1,10
1,1,Dorothea,Clayton,1,0,1,1,10
4,4,Matilda,Solomon,0,1,1,0,7
5,5,Kyllian,O'Berne,1,0,0,0,1
8,8,Guadalupe,Lebrun,0,0,0,1,4


In [14]:
elemTeach = teachers[teachers['Grade'] == "elem"]
msTeach = teachers[teachers['Grade'] == "ms"]

In [15]:
display(elemTeach.head())

,Teacher ID,Grade,Certificate,Relief
0,0,elem,1,6
1,1,elem,0,10
4,4,elem,1,10
5,5,elem,0,8
6,6,elem,0,7


In [16]:
display(msTeach.head())

,Teacher ID,Grade,Certificate,Relief
2,2,ms,1,5
3,3,ms,0,6
7,7,ms,1,9
8,8,ms,0,10
9,9,ms,0,8


In [17]:
students['Grade'].value_counts()
# If we were to divide each of these by 4, the number of classes, we would have about:
# 10 students per middle school class (so we'd like a range that encompasses 10 but has some wiggle room: 8 & 12)
# and 5 students per elementary class (so we'd like a range that encompasses 5 but has some wiggle room: 4 & 8)

Grade
ms      39
elem    21
Name: count, dtype: int64

In [18]:
num_classes = 4
e = cp.Variable((num_classes, len(elementary)), boolean = True)
m = cp.Variable((num_classes, len(middle)), boolean = True)
te = cp.Variable((num_classes, len(elemTeach)), boolean = True)
tm = cp.Variable((num_classes, len(msTeach)), boolean = True)

# Reference: https://github.com/samdavanloo/ISE-3230/blob/master/codes_in_CVXpy.ipynb : 3 Transportation
# Reference: https://www.statology.org/pandas-column-to-numpy-array/
E = elementary[['First Hour', 'Second Hour', 'Third Hour', 'Fourth Hour']].to_numpy()
M = middle[['First Hour', 'Second Hour', 'Third Hour', 'Fourth Hour']].to_numpy()
HE = elementary['Trouble'].to_numpy()
HM = middle['Trouble'].to_numpy()

constraints = []
for i in range(len(elementary)):
    # TODO: Need to figure out what's wrong with using cp.sum()
    constraints.append(cp.sum(E[i,:]*e[i,:]) == 1) # Each elementary student can attend only one class.
for k in range(len(middle)):
    constraints.append(cp.sum(M[i,:]*m[i,:]) == 1) # Each middle school student can attend only one class.
for j in range(num_classes):
    constraints.append(4 <= cp.sum(e[:,j]))
    constraints.append(cp.sum(e[:,j]) <= 8) # Number of students per elementary class must be between 4 & 8.
    # This is because we would like to have a ratio of about two to three students per one teacher at most.
    constraints.append(8 <= cp.sum(m[:,j]))
    constraints.append(cp.sum(m[:,j]) <= 12) # Number of students per middle school class must be between 8 & 12.
    # Again, we want the ratio to be about two to three students per one teacher at most.
# TODO: Add more constraints where needed.

# TODO: Figure out how to formulate the objective function here.

C:\Users\Morgan Eads\anaconda3\envs\env_opt\lib\site-packages\cvxpy\expressions\expression.py:650: UserWarning: 
This use of ``*`` has resulted in matrix multiplication.
Using ``*`` for matrix multiplication has been deprecated since CVXPY 1.1.
    Use ``*`` for matrix-scalar and vector-scalar multiplication.
    Use ``@`` for matrix-matrix and matrix-vector multiplication.
    Use ``multiply`` for elementwise multiplication.
This code path has been hit 1 times so far.

  warnings.warn(msg, UserWarning)


ValueError: Incompatible dimensions (1, 4) (21, 1)